In [80]:
from tensorflow.python.keras.models import load_model
from tensorflow.keras.applications import InceptionResNetV2, ResNet50
keras_model = ResNet50(weights="RadImageNet-ResNet50_notop.h5", input_shape=(256, 256, 3), include_top=False,pooling='avg')

from rad_networks import ResNet, block
torch_model = ResNet(block, num_classes=2)

In [85]:
keras_names, keras_params = [], []
for layer in keras_model.layers:
    for var in (layer.weights):
        splitted = var.name.split("_")
        if len(splitted) > 3:
            if splitted[2] == "0":
                continue
            else:
                keras_names.append(var.name)
                keras_params.append(var.numpy())
        else:
            keras_names.append(var.name)
            keras_params.append(var.numpy())

In [89]:
'''
 'conv1.weight',
 'conv1.bias',
 'bn1.weight',
 'bn1.bias',
 'bn1.running_mean',
 'bn1.running_var',
ここまでは手動で，あとは自動化したい．
torchモデルには，identityのbnがない
'''

import torch
torch_params = torch_model.state_dict()
torch_params_pop = torch_params.copy()

for i in range(len(list(torch_params.keys()))):
    if "tracked" in list(torch_params.keys())[i] or "identity" in list(torch_params.keys())[i]:
        torch_params_pop.pop(list(torch_params.keys())[i])

torch_names = list(torch_params_pop.keys())


torch_params["conv1.weight"].data = torch.tensor(keras_params[0].transpose(3,2,0,1))
torch_params["conv1.bias"].data = torch.tensor(keras_params[1])
torch_params["bn1.weight"].data = torch.tensor(keras_params[2])
torch_params["bn1.bias"].data = torch.tensor(keras_params[3])
torch_params["bn1.running_mean"].data = torch.tensor(keras_params[4])
torch_params["bn1.running_var"].data = torch.tensor(keras_params[5])


for i, keras_name in enumerate(keras_names):
    if "kernel" in keras_name:
        torch_params[torch_names[i]].data = torch.tensor(keras_params[i].transpose(3,2,0,1))
        layer_num += 1
    else:
        torch_params[torch_names[i]].data = torch.tensor(keras_params[i])
        layer_num += 1

torch_model.load_state_dict(torch_params)

<All keys matched successfully>

In [92]:
torch.save(torch_model.state_dict(), "RadImageNet_ResNet50_torch.pth")

In [44]:
layer_num = list(torch_params.keys())
for i in range(len(layer_num)):
    if "tracked" in layer_num[i]:
        torch_params.pop(layer_num[i])

In [62]:
# len(list(torch_params.keys())) 302

In [65]:
torch_names = list(torch_params.keys())
for i in range(len(torch_names)):
    if "tracked" in torch_names[i]:
        torch_params.pop(torch_names[i])

In [14]:
for layer in keras_model.layers:
    for var in (layer.weights):
        print(var.name, var.numpy().shape)

conv1_conv/kernel:0 (7, 7, 3, 64)
conv1_conv/bias:0 (64,)
conv1_bn/gamma:0 (64,)
conv1_bn/beta:0 (64,)
conv1_bn/moving_mean:0 (64,)
conv1_bn/moving_variance:0 (64,)
conv2_block1_1_conv/kernel:0 (1, 1, 64, 64)
conv2_block1_1_conv/bias:0 (64,)
conv2_block1_1_bn/gamma:0 (64,)
conv2_block1_1_bn/beta:0 (64,)
conv2_block1_1_bn/moving_mean:0 (64,)
conv2_block1_1_bn/moving_variance:0 (64,)
conv2_block1_2_conv/kernel:0 (3, 3, 64, 64)
conv2_block1_2_conv/bias:0 (64,)
conv2_block1_2_bn/gamma:0 (64,)
conv2_block1_2_bn/beta:0 (64,)
conv2_block1_2_bn/moving_mean:0 (64,)
conv2_block1_2_bn/moving_variance:0 (64,)
conv2_block1_0_conv/kernel:0 (1, 1, 64, 256)
conv2_block1_0_conv/bias:0 (256,)
conv2_block1_3_conv/kernel:0 (1, 1, 64, 256)
conv2_block1_3_conv/bias:0 (256,)
conv2_block1_0_bn/gamma:0 (256,)
conv2_block1_0_bn/beta:0 (256,)
conv2_block1_0_bn/moving_mean:0 (256,)
conv2_block1_0_bn/moving_variance:0 (256,)
conv2_block1_3_bn/gamma:0 (256,)
conv2_block1_3_bn/beta:0 (256,)
conv2_block1_3_bn/moving_

In [47]:
#for param in torch_model.named_parameters():
#    print(param[0], param[1].detach().numpy().shape)